## 1. Producing the data


### 1.1Process Event Producer

The code load  all the data from “Streaming_Linux_process.csv”. This will send X (random number between 10 and 50 inclusive) number of records from each machine following the sequence to the Kafka stream every 5 seconds. Event time is appended for each data in unix-timestamp format.


In [ ]:
# import statements
from time import sleep
from json import dumps
from kafka import KafkaProducer
import random
import datetime as dt
import csv


def read_process_data(fileName):
    
    #initialise arrays for each machine types
    machine4 = []
    machine5 = []
    machine6 = []
    machine7 = []
    machine8 = []
    
    #read process data from csv file
    with open(fileName) as file:
        reader = csv.DictReader(file)
        data = reader
        machine4 = [item for item in data if item['machine'] == '4' ]
        machine5 = [item for item in data if item['machine'] == '5' ]
        machine6 = [item for item in data if item['machine'] == '6' ]
        machine7 = [item for item in data if item['machine'] == '7' ]
        machine8 = [item for item in data if item['machine'] == '8' ]


    return {'machine4': machine4, 'machine5': machine5, 'machine6': machine6, 'machine7': machine7, 'machine8': machine8 }

def publish_message(producer_instance, topic_name, data):
    try:
        producer_instance.send(topic_name, data)
#         print('Message published successfully. Data: ' + str(data))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))
        
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  value_serializer=lambda x: dumps(x).encode('ascii'),
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer
    
if __name__ == '__main__':
   
    topic = 'process'
    processRows = read_process_data('Streaming_Linux_process.csv')
    print('Publishing records..')
    producer = connect_kafka_producer()
    
    #initialise the index value of data corresponding to each machine to 0
    start4 = 0
    start5 = 0
    start6 = 0
    start7 = 0
    start8 = 0
    
    def setMachineTime(start, r, machine, dic):
        
        #set end index of machine as start index + random number generated for the machine
        end = start+r
        
        #start from the beginning if the sequence is terminated
        if end >= len(processRows[machine]):
            start = 0
            end = r
            
        #add event time for all entries
        for item in processRows[machine][start:end]:
            item['ts'] = ts
            
        #append the data for machine 4 for the selected indices
        dic['data'].extend(processRows[machine][start:end])
        
        #update new start index with the current end index
        start = end
        
        return [start, end]
        
    while True:
        
        #get event time in the producer in unix format
        ts = str(dt.datetime.utcnow().strftime("%s"))
        dic = {'data': []}
        
        #initialise random numbers for every machines
        r4 = random.randint(10,50)   
        r5 = random.randint(10,50)   
        r6 = random.randint(10,50)   
        r7 = random.randint(10,50)   
        r8 = random.randint(10,50) 

        [start4, end4] = setMachineTime(start4, r4, 'machine4', dic)
        [start5, end5] = setMachineTime(start5, r5, 'machine5', dic)
        [start6, end6] = setMachineTime(start6, r6, 'machine6', dic)
        [start7, end7] = setMachineTime(start7, r7, 'machine7', dic)
        [start8, end8] = setMachineTime(start8, r8, 'machine8', dic)
        

        #publish the message with appropriate data
        publish_message(producer, topic, dic['data'])
        
        #send the sequence for every 5 seconds
        sleep(5)
   
        